# Tiles

> Fill in a module description here

In [ ]:
#| default_exp tiles

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import itertools
from pathlib import Path
from typing import List, Tuple, Dict
import cv2

import numpy as np
import tifffile as tiff
from scipy.optimize import linear_sum_assignment


def count(
    image: np.ndarray,
    t: int = 1024,
    v: float = 0.1
) -> Tuple[int, int]:
    """
    Calculate the number of tiles in the vertical and horizontal directions.

    Parameters
    ----------
    image : np.ndarray
        Input image array.
    t : int, optional
        Length (in pixels) of each square tile. Default is 1024.
    v : float, optional
        Fractional overlap between tiles (e.g., 0.1 for 10%). Default is 0.1.

    Returns
    -------
    n_tiles_X : int
        Number of tiles along the vertical axis (height).
    n_tiles_Y : int
        Number of tiles along the horizontal axis (width).
    """
    X = image.shape[0]
    Y = image.shape[1]
    n_tiles_X = round(((X / t) - v) / (1 - v))
    n_tiles_Y = round(((Y / t) - v) / (1 - v))
    return n_tiles_X, n_tiles_Y


def split(
    image: np.ndarray,
    tile_size: Tuple[int, int],
    overlap: int,
    tile_all: bool = True
) -> Tuple[np.ndarray, List[int], List[int]]:
    """
    Split an image into tiles of specified size with a given overlap.

    Parameters
    ----------
    image : np.ndarray
        Input image array (2D grayscale or 3D color).
    tile_size : tuple of int
        Height and width of each tile, e.g., (128, 128).
    overlap : int
        Number of pixels each tile overlaps adjacent tiles.
    tile_all : bool, optional
        If True, include partial edge tiles to cover the entire image. Default is True.

    Returns
    -------
    tiles : np.ndarray
        Array of shape (N, tile_h, tile_w[, C]) containing all tiles.
    x_pos : list of int
        X-coordinates of the top-left corner of each tile.
    y_pos : list of int
        Y-coordinates of the top-left corner of each tile.
    """
    img_height, img_width = image.shape[:2]
    tile_h, tile_w = tile_size
    step_h = tile_h - overlap
    step_w = tile_w - overlap

    row_starts = list(range(0, img_height - tile_h + 1, step_h))
    col_starts = list(range(0, img_width - tile_w + 1, step_w))
    if tile_all:
        if row_starts[-1] != img_height - tile_h:
            row_starts.append(img_height - tile_h)
        if col_starts[-1] != img_width - tile_w:
            col_starts.append(img_width - tile_w)

    tiles = []
    x_pos = []
    y_pos = []
    for y in row_starts:
        for x in col_starts:
            tiles.append(image[y:y + tile_h, x:x + tile_w])
            x_pos.append(x)
            y_pos.append(y)

    return np.array(tiles), x_pos, y_pos


def save(
    tiles: List[np.ndarray],
    filename: str,
    x: List[int],
    y: List[int],
    output_dir: str
) -> None:
    """
    Save a list of image tiles to disk with filenames encoding their positions.

    Parameters
    ----------
    tiles : list of np.ndarray
        List of image tiles to save.
    filename : str
        Original filename to derive base name for tiles.
    x : list of int
        X-coordinates corresponding to each tile.
    y : list of int
        Y-coordinates corresponding to each tile.
    output_dir : str
        Directory to save the tile files.

    Returns
    -------
    None
    """
    base = Path(filename).stem
    out_dir = Path(output_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    for idx, tile in enumerate(tiles):
        tile_fn = f"{base}_tile_{idx:04d}_x_{x[idx]}_y_{y[idx]}.tif"
        tiff.imwrite(str(out_dir / tile_fn), tile)


def stitch(
    tiles_list: List[np.ndarray],
    x_list: List[int],
    y_list: List[int],
    iou_threshold: float
) -> np.ndarray:
    """
    Stitch tile masks into a global mask using IoU-based label matching.

    Parameters
    ----------
    tiles_list : list of np.ndarray
        List of 2D mask arrays for each tile.
    x_list : list of int
        X-coordinates of tiles' top-left corners.
    y_list : list of int
        Y-coordinates of tiles' top-left corners.
    iou_threshold : float
        Minimum IoU threshold to match cell labels between overlapping tiles.

    Returns
    -------
    global_mask : np.ndarray
        Stitched mask with consistent global labels.
    """
    tiles = _prepare_tiles(tiles_list, x_list, y_list)
    global_mask = _init_global_mask(tiles)
    next_label = 1

    global_mask, next_label = _assign_first_tile(tiles[0], global_mask, next_label)
    for tile in tiles[1:]:
        global_mask, next_label = _process_tile(tile, global_mask, iou_threshold, next_label)

    return global_mask


def _prepare_tiles(
    masks: List[np.ndarray],
    xs: List[int],
    ys: List[int]
) -> List[Dict[str, np.ndarray]]:
    """
    Package raw masks and coordinates into a uniform tile dict format.

    Parameters
    ----------
    masks : list of np.ndarray
        List of tile mask arrays.
    xs : list of int
        X-coordinates of each tile.
    ys : list of int
        Y-coordinates of each tile.

    Returns
    -------
    tiles : list of dict
        List of dictionaries each with keys 'mask', 'x', 'y'.
    """
    return [
        {"mask": m, "x": x, "y": y}
        for m, x, y in zip(masks, xs, ys)
    ]


def _init_global_mask(
    tiles: List[Dict[str, np.ndarray]]
) -> np.ndarray:
    """
    Initialize the global mask array based on tile extents.

    Parameters
    ----------
    tiles : list of dict
        List of tile dictionaries with 'mask', 'x', 'y'.

    Returns
    -------
    global_mask : np.ndarray
        Zero-initialized mask sized to hold all tiles.
    """
    max_y = max(tile["y"] + tile["mask"].shape[0] for tile in tiles)
    max_x = max(tile["x"] + tile["mask"].shape[1] for tile in tiles)
    return np.zeros((max_y, max_x), dtype=np.int32)


def _assign_first_tile(
    tile: Dict[str, np.ndarray],
    global_mask: np.ndarray,
    next_label: int
) -> Tuple[np.ndarray, int]:
    """
    Assign unique global labels to the first tile and paste into global mask.

    Parameters
    ----------
    tile : dict
        Dictionary with keys 'mask', 'x', 'y' for the first tile.
    global_mask : np.ndarray
        The initialized global mask array.
    next_label : int
        Next available global label index.

    Returns
    -------
    global_mask : np.ndarray
        Updated global mask with first tile pasted.
    next_label : int
        Updated next available global label index.
    """
    mask = tile["mask"]
    y0, x0 = tile["y"], tile["x"]
    mapping = {}

    for lab in np.unique(mask):
        if lab == 0:
            continue
        mapping[lab] = next_label
        next_label += 1

    remapped = _remap_tile(mask, mapping)
    h, w = mask.shape
    global_mask[y0:y0 + h, x0:x0 + w] = remapped
    return global_mask, next_label


def _process_tile(
    tile: Dict[str, np.ndarray],
    global_mask: np.ndarray,
    iou_threshold: float,
    next_label: int
) -> Tuple[np.ndarray, int]:
    """
    Match and paste a subsequent tile into the global mask.

    Parameters
    ----------
    tile : dict
        Dictionary with keys 'mask', 'x', 'y' for the tile.
    global_mask : np.ndarray
        Current global mask array.
    iou_threshold : float
        IoU threshold for label matching.
    next_label : int
        Next available global label index.

    Returns
    -------
    global_mask : np.ndarray
        Updated global mask with new tile merged.
    next_label : int
        Updated next available global label index.
    """
    mask = tile["mask"]
    y0, x0 = tile["y"], tile["x"]
    h, w = mask.shape

    region = global_mask[y0:y0 + h, x0:x0 + w].copy()
    mapping, next_label = _build_label_mapping(mask, region, iou_threshold, next_label)
    remapped = _remap_tile(mask, mapping)

    update_mask = (region == 0) & (remapped != 0)
    region[update_mask] = remapped[update_mask]
    global_mask[y0:y0 + h, x0:x0 + w] = region

    return global_mask, next_label


def _build_label_mapping(
    mask: np.ndarray,
    region: np.ndarray,
    iou_threshold: float,
    next_label: int
) -> Tuple[Dict[int, int], int]:
    """
    Compute a mapping from tile labels to global labels based on IoU.

    Parameters
    ----------
    mask : np.ndarray
        Tile mask array.
    region : np.ndarray
        Corresponding region in the global mask.
    iou_threshold : float
        Minimum IoU threshold for matching.
    next_label : int
        Next available global label index.

    Returns
    -------
    mapping : dict of int to int
        Mapping from tile labels to global labels.
    next_label : int
        Updated next available global label index.
    """
    overlap = (mask != 0) & (region != 0)
    tile_labels = np.unique(mask[overlap])
    global_labels = np.unique(region[overlap])
    mapping: Dict[int, int] = {}

    if tile_labels.size and global_labels.size:
        iou_mat = np.zeros((len(tile_labels), len(global_labels)))
        for i, tl in enumerate(tile_labels):
            tl_mask = (mask == tl)
            for j, gl in enumerate(global_labels):
                gl_mask = (region == gl)
                inter = np.sum(tl_mask & gl_mask)
                union = np.sum(tl_mask | gl_mask)
                iou_mat[i, j] = inter / union if union else 0.0

        cost = -iou_mat
        rows, cols = linear_sum_assignment(cost)
        for r, c in zip(rows, cols):
            if iou_mat[r, c] >= iou_threshold:
                mapping[tile_labels[r]] = global_labels[c]

    for lab in np.unique(mask):
        if lab and lab not in mapping:
            mapping[lab] = next_label
            next_label += 1

    return mapping, next_label


def _remap_tile(
    mask: np.ndarray,
    mapping: Dict[int, int]
) -> np.ndarray:
    """
    Remap tile mask labels using a lookup array for performance.

    Parameters
    ----------
    mask : np.ndarray
        Input tile mask.
    mapping : dict of int to int
        Mapping from old to new labels.

    Returns
    -------
    np.ndarray
        Remapped tile mask array.
    """
    max_old = mask.max()
    lookup = np.zeros(max_old + 1, dtype=np.int32)
    for old, new in mapping.items():
        lookup[old] = new
    return lookup[mask]
